# How to train a agent for ML-Agents-SoccerTwos

## Configurations

In [1]:
%%capture
# Clone the repository
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

You need to run this twice

In [ ]:
%cd /content/ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

Create the folder "train-soccer"

In [ ]:
%cd /content/ml-agents
!mkdir -p ./train-soccer

Download the zip file of the training environment executable and unzip it

In [ ]:
import gdown
file_id = '1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', './train-soccer/SoccerTwos.zip', quiet=False)

In [4]:
%%capture
!unzip -d ./train-soccer/SoccerTwos ./train-soccer/SoccerTwos.zip

Change the permissions of the file

In [5]:
!chmod -R 755 /content/ml-agents/train-soccer/SoccerTwos/SoccerTwos.x86_64

## Train from scratch

Create the YAML file with the agent parameters. You can change them as you like

In [19]:
yaml_content = """
behaviors:
  SoccerTwos:
    trainer_type: poca
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: constant
    network_settings:
      normalize: false
      hidden_units: 512
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    keep_checkpoints: 5
    max_steps: 5000000
    time_horizon: 1000
    summary_freq: 10000
    self_play:
      save_steps: 50000
      team_change: 200000
      swap_steps: 2000
      window: 10
      play_against_latest_model_ratio: 0.5
      initial_elo: 1200.0
"""

with open('./train-soccer/SoccerTwos.yaml', 'w') as file:
    file.write(yaml_content)

Train the agent

Tip: You can stop the training at any time. If you want to resume it later, go to 'Continue training'.

In [ ]:
!mlagents-learn ./train-soccer/SoccerTwos.yaml --env=train-soccer/SoccerTwos/SoccerTwos.x86_64 --run-id="SoccerTwosNew" --no-graphics --force

## Push the trained agent to a Hugging Face repository.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Change the `--repo-id` to your username, for example: `huggingFaceUser/ML-Agents-SoccerTwos`.

In [ ]:
!mlagents-push-to-hf --run-id="SoccerTwosNew" --local-dir="./results/SoccerTwosNew" --repo-id="huggingFaceUser/ML-Agents-SoccerTwos" --commit-message="initial commit"

## Continue training

If you have already trained the agent above and created the repository, but still have the files, skip this part and go to 'Resume the agent's training'. If the environment files have been reset and the repository is created, continue.

Remember to run the configurations part

In [ ]:
!git clone https://huggingface.co/huggingFaceUser/ML-Agents-SoccerTwos

In [7]:
!mkdir -p ./results/SoccerTwosNew
!mv ./ML-Agents-SoccerTwos/* ./results/SoccerTwosNew/

### Resume the agent's training

In [ ]:
!mlagents-learn ./results/SoccerTwosNew/configuration.yaml --env=train-soccer/SoccerTwos/SoccerTwos.x86_64 --run-id="SoccerTwosNew" --no-graphics --resume

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!mlagents-push-to-hf --run-id="SoccerTwosNew" --local-dir="./results/SoccerTwosNew" --repo-id="huggingFaceUser/ML-Agents-SoccerTwos" --commit-message="Agent training resumed from the checkpoint."